In [4]:
import cv2
import numpy as np

# Load YOLOv3 model and its configuration
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load class names
classes = []
with open("coco.txt", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get the output layer names from the YOLO model
layer_names = net.getLayerNames()

# Handle different OpenCV versions for getting output layers
try:
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
except IndexError:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Generate random colors for each class
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# List of image paths to process
image_paths = [
    "1.jpg",  # Replace with your image paths
    "2.jpg",
    # Add more paths as needed
]

# Process each image in the list
for image_path in image_paths:
    # Load the image
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Could not load image {image_path}")
        continue

    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for detected objects' details
    class_ids = []
    confidences = []
    boxes = []

    # Process the outputs from the network
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Perform non-maximum suppression to eliminate redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.3)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

    # Save and/or display the output image
    output_path = f"output_{image_path.split('/')[-1]}"  # Save with a unique name
    cv2.imwrite(output_path, frame)
    cv2.imshow("Object Detection", frame)
    cv2.waitKey(0)  # Wait for a key press before moving to the next image
    cv2.destroyAllWindows()

print("Processing completed for all images.")


Processing completed for all images.
